In [6]:
# -*- coding: utf-8 -*-

#  Licensed under the Apache License, Version 2.0 (the "License"); you may
#  not use this file except in compliance with the License. You may obtain
#  a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
#  WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
#  License for the specific language governing permissions and limitations
#  under the License.

import os
import sys
import wsgiref.simple_server as simple_server
from argparse import ArgumentParser

from builtins import bytes
from linebot import (
    LineBotApi, WebhookParser
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage
)
from linebot.utils import PY3

In [7]:
# get channel_secret and channel_access_token from your environment variable
channel_secret = 'c73718e5e705e7be5c3736c49c4863b5'
channel_access_token = 'MW/mgLewP4q1ko1rDC2hP4+RkKfnM3BPNK7yj0vT7sDwUJh2Wnb3ixSnHVwcpWgNTSpVuTRW5dUf9sXYhUg0MPdJX31PPMDiAsacCk+fy1bzQhUBkIsXe2TYKTMJdrpZM5QcD5iupo3Byjt+nyCEZgdB04t89/1O/w1cDnyilFU='
if channel_secret is None:
    print('Specify LINE_CHANNEL_SECRET as environment variable.')
    sys.exit(1)
if channel_access_token is None:
    print('Specify LINE_CHANNEL_ACCESS_TOKEN as environment variable.')
    sys.exit(1)

line_bot_api = LineBotApi(channel_access_token)
parser = WebhookParser(channel_secret)

In [18]:
file_path = './received.jpeg'
def application(environ, start_response):
    # check request path
    path_info = environ['PATH_INFO']
    if path_info != '/callback':
        print(path_info)
        if path_info.lower().find('images') != 1:
            start_response('404 Not Found', [])
            return create_body('Not Found')
    if path_info == '/callback':
        # check request method
        if environ['REQUEST_METHOD'] != 'POST':
            start_response('405 Method Not Allowed', [])
            return create_body('Method Not Allowed')

        # get X-Line-Signature header value
        signature = environ['HTTP_X_LINE_SIGNATURE']

        # get request body as text
        wsgi_input = environ['wsgi.input']
        content_length = int(environ['CONTENT_LENGTH'])
        body = wsgi_input.read(content_length).decode('utf-8')

        # parse webhook body
        try:
            events = parser.parse(body, signature)
        except InvalidSignatureError:
            start_response('400 Bad Request', [])
            return create_body('Bad Request')

        # if event is MessageEvent and message is TextMessage, then echo text
        for event in events:
            if not isinstance(event, MessageEvent):
                continue
            if not isinstance(event.message, TextMessage) and not isinstance(event.message, ImageMessage):
                continue
            if isinstance(event.message, TextMessage):        
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text=event.message.text)
                )
            else:
                message_content = line_bot_api.get_message_content(event.message.id)
                with open(file_path, 'wb') as fd:
                    for chunk in message_content.iter_content():
                        fd.write(chunk)
                    fd.close()
                #copy file_path to https://drive.google.com/drive/folders/1B6yZwKZXOhDPkdl8st7_IYdUGZCKfVYp?usp=sharing

                echoImageText = ImageSendMessage(
                  original_content_url = "https://835d-161-200-85-86.ap.ngrok.io/images/mos.jpg",
                  preview_image_url = "https://835d-161-200-85-86.ap.ngrok.io/images/mos.jpg"
                )
                line_bot_api.reply_message(
                    event.reply_token,
                    echoImageText
                )
        start_response('200 OK', [])
        return create_body('OK')    
                
    else:
        headers = [('Content-type', 'image/jpeg')]
        start_response('200 OK', headers)
        data = b''
        filename = file_path
        with open(filename, 'rb', buffering=0) as f:
            data = f.readall()
        print(type(data))       #<class 'bytes'>
        return [data]    

In [19]:
def create_body(text):
    if PY3:
        return [bytes(text, 'utf-8')]
    else:
        return text

In [20]:
if __name__ == '__main__':
#     arg_parser = ArgumentParser(
#         usage='Usage: python ' + __file__ + ' [--port <port>] [--help]'
#     )
#     arg_parser.add_argument('-p', '--port', type=int, default=5000, help='port')
#     options = arg_parser.parse_args()

    path = 'public' 
    port = 5000
    httpd = simple_server.make_server('', port, application)
    print("Serving {} on port {}, control-C to stop".format(path, port))
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("Shutting down.")
        httpd.server_close()

Serving public on port 5000, control-C to stop


127.0.0.1 - - [22/Jul/2022 14:00:27] "POST /callback HTTP/1.1" 200 2


/images/mos.jpg
<class 'bytes'>


127.0.0.1 - - [22/Jul/2022 14:00:28] "GET /images/mos.jpg HTTP/1.1" 200 172380
127.0.0.1 - - [22/Jul/2022 14:00:56] "POST /callback HTTP/1.1" 200 2


/images/mos.jpg
<class 'bytes'>


...

/images/mos.jpg
<class 'bytes'>


127.0.0.1 - - [22/Jul/2022 14:33:37] "GET /images/mos.jpg HTTP/1.1" 200 734731


/images/mos.jpg
<class 'bytes'>


127.0.0.1 - - [23/Jul/2022 00:21:39] "GET /images/mos.jpg HTTP/1.1" 200 734731


Shutting down.
